# 벽돌 데이터 수집

## 코드 설계

 IMAGE --> grayscale --> threshold--> erode --> find contour --> cut --> normalization

## 파일 불러오기 및 준비

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

# generator 객체를 반환.
def readFiles(path, opener=cv2.imread):
    filepath_list = []
    file_list = []

    if not os.path.isdir(path):
        raise Exception(f"'{path}' is not a directory.")

    for f in os.listdir(path):
        filepath = os.path.join(path, f)

        if os.path.isfile(filepath):
            filepath_list.append(filepath)
            file_list.append(opener(filepath))

    return filepath_list, file_list

## plot에서 한글이 출력 안되는 문제 해결

In [ ]:
import matplotlib.font_manager as fm

korean_font = fm.FontProperties(fname='/System/Library/Fonts/AppleSDGothicNeo.ttc', size=14)

## 파손 침식 등을 검출

In [ ]:
# 파손, 침식된 부분만 남기고 삭제
def threshold(frame_gray):
    # sobelX = cv2.Sobel(frame, cv2.CV_64F, 1, 0, ksize=3)
    # sobelX = cv2.convertScaleAbs(sobelX)
    # frame = sobelX

    # kernel = np.array([
    #     [0, 0, 0, 0, 0],
    #     [0, 1, 1, 1, 0],
    #     [1, 1, 1, 1, 1],
    #     [0, 1, 1, 1, 0],
    #     [0, 0, 0, 0, 0]
    # ], dtype=np.uint8)
    # frame = cv2.erode(frame, kernel, iterations=1)

    # frame = cv2.adaptiveThreshold(frame, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 9, 1)
    frame_bin = cv2.threshold(frame_gray, 64, 255, cv2.THRESH_BINARY_INV)[-1]
    return frame_bin

def locateBreaks(frame):
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_bin = threshold(frame_gray)

    contours = cv2.findContours(frame_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    def _(cont):
        x,y,w,h = cv2.boundingRect(cont)
        cx, cy = (x + w//2, y + h//2)
        return {
            'x': x,
            'y': y,
            'w': w,
            'h': h,
            'cx': cx,
            'cy': cy,
            'contour': cont
        }
    return [ _(c) for c in contours ]


In [ ]:
def findHoles(frame_bgr):
    frame_hsv = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2HSV)
    # 채도(S)와 밝기(V)가 낮은 영역 : 검은색 영역을 선택
    frame_bin = cv2.inRange(frame_hsv, (  0,  0,  0), (255, 96, 96)) 
    return frame_bin

def findContoursFromMask(frame_color, mask):
    contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    for cont in contours:
        if cv2.contourArea(cont) < 10: continue

        x, y, w, h = cv2.boundingRect(cont)
        cx, cy = x + w/2, y + h/2
        n = max(w,h)
        #
        x0, y0 = int(cx - n/2), int(cy - n/2)
        x1, y1 = int(cx + n/2), int(cy + n/2)
        #
        frame_cut = np.zeros((n,n,3), dtype=np.uint8)
        frame_cut = frame_color[y0:y1, x0:x1]
        #
        yield frame_cut.copy()


## 결과 출력

In [ ]:
filepath_list, frame_list = readFiles('./res/', opener=cv2.imread)

for filepath, frame_bgr in zip(filepath_list, frame_list):
    if (type(frame_bgr) == type(None)) : continue

    print(f"For file '{filepath}':")

    mask = findHoles(frame_bgr)
    frame_cut_list = list(findContoursFromMask(frame_bgr, mask))
    idx_list = range(len(frame_cut_list))

    for frame_cut, idx in zip(frame_cut_list, idx_list):
        cv2.imwrite(f'./out/{os.path.basename(filepath)}_{idx}.png', frame_cut)
        #
    plt.imshow(cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB))
    plt.show()
    plt.imshow(hole_mask, cmap='gray')
    plt.show()
    continue


    frame_out = image.copy()
    locations = locateBreaks(image)

    print(f"\nFor file '{filepath}':\n")
    for lc, idx in zip(locations, range(len(locations))):
        x,y,w,h,cx,cy,cont = [lc[x] for x in ['x','y','w','h','cx','cy','contour']]


        n = max(w,h)
        c = n / 2

        cv2.rectangle(frame_out, (x,y), (x+w, y+h), (0,0,255), 1)


        cv2.imwrite(f'./out/{os.path.basename(filepath)}_{idx}', image[ lc['y']:lc['y']+lc['h'], lc['x']:lc['x']+lc['w'] ])
        print(f"\r {filepath}_{idx}", end='')
        continue

        partial = np.zeros((n,n,3), dtype=np.uint8)

        x0,y0 = max(  0, cx-c), max(  0, cy-c)
        x1,y1 = min(256, cx+c), min(256, cy+c)

        x0,y0,x1,y1 = [round(x) for x in (x0,y0,x1,y1)]
        
        cut = image[y0:y1, x0:x1]

        h, w = y1-y0, x1-x0

        x0,y0 = max(  0, c-h/2), max(  0, c+h/2)
        x1,y1 = min(256, c-w/2), min(256, c-w/2)

        x0,y0,x1,y1 = [round(x) for x in (x0,y0,x1,y1)]

        partial[y0:y1, x0:x1] = cut

        cv2.imwrite(f'{filepath}_{i}', partial)

    plt.imshow(cv2.cvtColor(frame_out, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
type(None)